In [4]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
from IPython.display import display
import datetime

mypath = r"C:\Users\justr\Documents\dci-2024\raw data"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for f in files:
    print(f)

20240626rockford.htm
20240628muncie.htm
20240628walnut.htm
20240629madison.htm
20240629shelton.htm
20240629walnut.htm
20240630lisle.htm
20240630sanbernardino.htm
20240702mason.htm
20240703cedarburg.htm
20240705clovis.htm
20240705portland.htm
20240705rockford.htm
20240706seattle.htm
20240706whitewater.htm


In [5]:
scores_list = list()

scores_table = pd.DataFrame(np.zeros([1,18]))
scores_table.columns = ['Corps','Date','Location','Performance Slot','General Effect 1','General Effect 2','General Effect Total','Visual Proficiency','Visual Analysis','Color Guard','Visual Total','Music Brass','Music Analysis','Music Percussion','Music Total','Sub Total','Penalty','Total Score']

for (show_number, file) in enumerate(files):

    # print(file)

    df_list = pd.read_html("raw data/"+file)
    df = df_list[-1]

    date_and_place = df.iloc[0, 0]
    # print(date_and_place)
    chunks = date_and_place.split(sep=' ')
    place = " ".join(chunks[1:])

    for (i, r) in enumerate(df[0]):
        if r in ['DCI World Class', 'DCI Open Class', 'DCI All Age  Class A','DCI  All Age Open Class','DCI All Age  Open Class','DCI  All Age World Class', 'DCI  All Age Class A']:
            df.drop(i, inplace=True)

    df = df.iloc[4:-1].reset_index(drop=True)

    # grab names
    names = df[0].reset_index(drop=True)

    clean_names = list()
    for (i, r) in enumerate(names):
        if (i % 2) == 0:
            running_name = str(r)
        else:
            try:
                float(r)
            except:
                running_name += " " + r
            clean_names.append(running_name)
    # print(clean_names)

    df = df.iloc[::2, :].reset_index(drop=True)
    df.columns = ['Corps','Performance Slot','General Effect 1','General Effect 2','General Effect Total','Visual Proficiency','Visual Analysis','Color Guard','Visual Total','Music Brass','Music Analysis','Music Percussion','Music Total','Sub Total','Penalty','Total Score']
    df['Corps'] = clean_names
    df.insert(1,column='Date',value=pd.to_datetime(chunks[0]))
    df.insert(2,column='Location',value=place)
    df.insert(0,column='Show Number',value=show_number+1)
    df.iloc[:,5:] = df.iloc[:,5:].astype(float)
    df['Performance Slot'] = df['Performance Slot'].astype(int)

    scores_list.append(df)

scores_table = pd.concat(scores_list)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(scores_table)

,Show Number,Corps,Date,Location,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,Color Guard,Visual Total,Music Brass,Music Analysis,Music Percussion,Music Total,Sub Total,Penalty,Total Score
0,1,Boston Crusaders,2024-06-26,"Rockford, Michigan",5,15.4,14.4,29.8,14.5,14.3,14.5,21.65,14.3,14.7,14.1,21.55,73.0,0.0,73.0
1,1,Phantom Regiment,2024-06-26,"Rockford, Michigan",2,14.8,14.2,29.0,14.0,13.6,13.6,20.6,13.8,14.2,13.1,20.55,70.15,0.0,70.15
2,1,Cavaliers,2024-06-26,"Rockford, Michigan",4,13.7,13.9,27.6,13.8,13.3,12.9,20.0,13.4,14.1,13.8,20.65,68.25,0.0,68.25
3,1,Blue Stars,2024-06-26,"Rockford, Michigan",1,14.1,13.3,27.4,13.3,12.8,13.4,19.75,12.9,13.6,12.5,19.5,66.65,0.0,66.65
4,1,Colts,2024-06-26,"Rockford, Michigan",3,13.8,13.0,26.8,13.4,12.6,12.8,19.4,12.7,13.3,12.4,19.2,65.4,0.0,65.4
0,2,Carolina Crown,2024-06-28,"Muncie, Indiana",6,14.9,15.4,30.3,15.2,15.0,14.6,22.4,15.1,15.1,14.2,22.2,74.9,0.0,74.9
1,2,Boston Crusaders,2024-06-28,"Muncie, Indiana",5,15.0,15.0,30.0,14.7,15.2,14.9,22.4,14.8,14.8,15.0,22.3,74.7,0.0,74.7
2,2,Phantom Regiment,2024-06-28,"Muncie, Indiana",3,14.5,14.7,29.2,14.5,14.2,13.8,21.25,14.4,14.7,14.3,21.7,72.15,0.0,72.15
3,2,Cavaliers,2024-06-28,"Muncie, Indiana",2,13.7,14.1,27.8,14.0,13.4,13.1,20.25,13.7,14.3,14.1,21.05,69.1,0.0,69.1
4,2,Blue Stars,2024-06-28,"Muncie, Indiana",1,13.4,13.8,27.2,13.2,13.3,13.5,20.0,13.3,13.8,12.9,20.0,67.2,0.0,67.2


In [6]:
current_scores = dict()

for corps in set(scores_table['Corps']):
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    current_scores.update({corps: list(scores_table_corps['Total Score'])[-1]})

In [7]:
import plotly.graph_objects as go
import plotly.colors as pc

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold
for (i, (corps, current_score)) in enumerate(sorted(current_scores.items(), key=lambda x:x[1], reverse=True)):
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    fig.add_trace(go.Scatter(
        x=scores_table_corps['Date'],
        y=scores_table_corps['Total Score'],
        name=f'#{ranking} ({current_score:.2f}) {corps}',
        mode='lines+markers',
        connectgaps=True,
        line=dict(
            #shape='hv',
            color=trace_colors[i % len(trace_colors)]
        ),
        text=[
            f"Location: {x[0]}"
            for x in zip(
                scores_table_corps['Location']
            )
        ]
    ))
    ranking += 1

updated_time = f'<i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT</i>'

fig.update_layout(
    title=f'<b>DCI 2024 Scores by Roman Ramirez</b><br>{updated_time}<br>',
    xaxis_title='<b>Date</b>',
    yaxis_title='<b>Total Score</b>'
)

customdata = np.stack((
    list(scores_table['Location'])
    ), axis=-1)
hovertemplate = (
    '<i>%{x|%A, %B %d, %Y}</i><br>' + 
    '%{text}<br>' +
    '<b>%{fullData.name}</b><br>' + 
    '<b>Show\'s Total Score: </b>%{y}' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Current Score</i>) <i>Corps</i></b>'
)

fig.show()
fig.write_html("index.html")

In [8]:
#to dos

# in import, create "static" corps master file
# # contains class, and other corps-related info

# SQL join this on scores table
# add buttons to filter by class